<a href="https://colab.research.google.com/github/DAIVIKAD/Machine-Learning-AI-programs/blob/main/api_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Colab-ready script to list Groq models and test chat completions
# Save this cell and run. Requires 'requests' (preinstalled in Colab).

import os
import requests
import csv
import time
from getpass import getpass
from datetime import datetime
from IPython.display import display
import pandas as pd

# Config
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")  # prefer env var
if not GROQ_API_KEY:
    print("No GROQ_API_KEY found in environment. Please paste it (input is hidden):")
    GROQ_API_KEY = getpass("GROQ_API_KEY: ").strip()

HEADERS = {
    "Authorization": f"Bearer {GROQ_API_KEY}",
    "Content-Type": "application/json",
}

MODELS_ENDPOINT = "https://api.groq.com/openai/v1/models"
CHAT_ENDPOINT = "https://api.groq.com/openai/v1/chat/completions"

# Safety / run limits
MAX_TESTS = 25  # change if you want to test more models (cost & rate-limit considerations)
REQUEST_TIMEOUT = 30  # seconds
PAUSE_BETWEEN = 0.25  # seconds between requests to be polite

def get_models():
    r = requests.get(MODELS_ENDPOINT, headers=HEADERS, timeout=REQUEST_TIMEOUT)
    r.raise_for_status()
    data = r.json()
    # Many APIs return { "data": [ { "id": "model-name", ...}, ... ] }
    if isinstance(data, dict) and "data" in data and isinstance(data["data"], list):
        return [m.get("id") or m.get("model") or m for m in data["data"]]
    # fallback: maybe models returned as list
    if isinstance(data, list):
        return [m.get("id") if isinstance(m, dict) else m for m in data]
    # otherwise return raw
    return data

def test_model_chat(model_id):
    # Small, safe test prompt. Keeps response short by instructing a tiny structured reply.
    payload = {
        "model": model_id,
        "messages": [
            {"role": "system", "content": "You are a tiny test assistant that must reply with a single short line."},
            {"role": "user", "content": "Test connectivity. Reply exactly 'MODEL_OK' followed by the model id."}
        ],
        "max_tokens": 32,
        "temperature": 0.0
    }
    try:
        r = requests.post(CHAT_ENDPOINT, headers=HEADERS, json=payload, timeout=REQUEST_TIMEOUT)
        # capture status and body
        return {
            "ok": r.ok,
            "status_code": r.status_code,
            "response_json": r.json() if r.content else None,
            "error_text": None if r.ok else r.text[:1000]
        }
    except Exception as e:
        return {"ok": False, "status_code": None, "response_json": None, "error_text": str(e)}

def main():
    start_ts = datetime.utcnow().isoformat() + "Z"
    print(f"Started at {start_ts}")
    print("Fetching available models from Groq...")
    try:
        raw_models = get_models()
    except Exception as e:
        print("Failed to fetch models:", e)
        return

    # normalize model ids (if list of dicts etc)
    if not isinstance(raw_models, list):
        print("Unexpected models response (not a list). Here's the raw content:")
        print(raw_models)
        return

    model_ids = []
    for m in raw_models:
        if isinstance(m, str):
            model_ids.append(m)
        elif isinstance(m, dict):
            # common key names
            for k in ("id", "model", "name"):
                if k in m:
                    model_ids.append(m[k])
                    break
            else:
                # fallback serialize
                model_ids.append(str(m))

    print(f"Found {len(model_ids)} models. Showing up to MAX_TESTS={MAX_TESTS}.")
    to_test = model_ids[:MAX_TESTS]

    results = []
    for idx, mid in enumerate(to_test, start=1):
        print(f"[{idx}/{len(to_test)}] Testing model: {mid}", end=" ... ", flush=True)
        res = test_model_chat(mid)
        if res["ok"] and res["response_json"]:
            # try to pull generated text safely
            text = None
            try:
                # Groq chat response shape is 'choices' or 'output' depending on API version
                js = res["response_json"]
                # try common OpenAI-compatible shape
                choices = js.get("choices")
                if choices and isinstance(choices, list) and len(choices) > 0:
                    # prefer message content
                    c0 = choices[0]
                    if "message" in c0 and isinstance(c0["message"], dict):
                        text = c0["message"].get("content")
                    else:
                        text = c0.get("text") or c0.get("message")
                # fallback: 'text' or 'output' in top-level
                if not text:
                    text = js.get("text") or str(js.get("output") or js)
            except Exception:
                text = str(res["response_json"])
            print("OK")
        else:
            text = None
            print("FAILED")
        results.append({
            "model_id": mid,
            "ok": bool(res["ok"]),
            "status_code": res["status_code"],
            "short_response": (text[:400] if isinstance(text, str) else None),
            "error": res["error_text"]
        })
        time.sleep(PAUSE_BETWEEN)

    # Save results to CSV
    csv_path = "/content/groq_model_test_results.csv"
    with open(csv_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["model_id","ok","status_code","short_response","error"])
        writer.writeheader()
        for r in results:
            writer.writerow(r)
    print("Results written to:", csv_path)

    # show dataframe for quick inspection in Colab
    df = pd.DataFrame(results)
    display(df)
    print("Done. Inspect the CSV or DataFrame above. If you want to test all models, increase MAX_TESTS.")
    return results

if __name__ == "__main__":
    main()


No GROQ_API_KEY found in environment. Please paste it (input is hidden):
GROQ_API_KEY: ··········


/tmp/ipython-input-1030612624.py:69: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_ts = datetime.utcnow().isoformat() + "Z"


Started at 2025-11-12T13:36:40.469121Z
Fetching available models from Groq...
Found 20 models. Showing up to MAX_TESTS=25.
[1/20] Testing model: moonshotai/kimi-k2-instruct ... OK
[2/20] Testing model: qwen/qwen3-32b ... OK
[3/20] Testing model: openai/gpt-oss-20b ... OK
[4/20] Testing model: openai/gpt-oss-safeguard-20b ... OK
[5/20] Testing model: playai-tts-arabic ... FAILED
[6/20] Testing model: moonshotai/kimi-k2-instruct-0905 ... OK
[7/20] Testing model: whisper-large-v3 ... FAILED
[8/20] Testing model: openai/gpt-oss-120b ... OK
[9/20] Testing model: whisper-large-v3-turbo ... FAILED
[10/20] Testing model: groq/compound ... OK
[11/20] Testing model: groq/compound-mini ... OK
[12/20] Testing model: allam-2-7b ... OK
[13/20] Testing model: meta-llama/llama-guard-4-12b ... OK
[14/20] Testing model: playai-tts ... FAILED
[15/20] Testing model: meta-llama/llama-prompt-guard-2-86m ... FAILED
[16/20] Testing model: llama-3.3-70b-versatile ... OK
[17/20] Testing model: meta-llama/llama-

,model_id,ok,status_code,short_response,error
0,moonshotai/kimi-k2-instruct,True,200,MODEL_OK tiny-model,None
1,qwen/qwen3-32b,True,200,"<think>\nOkay, the user wants me to test conne...",None
2,openai/gpt-oss-20b,True,200,{'id': 'chatcmpl-0890c431-8096-460a-af1f-c580b...,None
3,openai/gpt-oss-safeguard-20b,True,200,{'id': 'chatcmpl-defc941a-3ca9-4168-bad4-e887d...,None
4,playai-tts-arabic,False,400,None,"{""error"":{""message"":""The model `playai-tts-ara..."
5,moonshotai/kimi-k2-instruct-0905,True,200,MODEL_OK tiny_test_assistant,None
6,whisper-large-v3,False,400,None,"{""error"":{""message"":""The model `whisper-large-..."
7,openai/gpt-oss-120b,True,200,{'id': 'chatcmpl-cfa93e4f-53ed-445e-805a-c1582...,None
8,whisper-large-v3-turbo,False,400,None,"{""error"":{""message"":""The model `whisper-large-..."
9,groq/compound,True,200,MODEL_OK Compound,None


Done. Inspect the CSV or DataFrame above. If you want to test all models, increase MAX_TESTS.
